In [ ]:
%matplotlib inline
import matplotlib
backend = matplotlib.get_backend()
import matplotlib.pyplot as plt

from pathlib import Path
from skimage import io
import numpy as np

import importlib
import os

# This line will supress the TF output 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf

# midap import and reset of backend to notebook
from midap.data.tf_pipeline import TFPipeFamilyMachine
matplotlib.use(backend)

## Model testing with MIDAP

This notebook can be used to test models. It calculates the accuracy of the model for a test set and makes some plots. 

In [ ]:
#########################
# define the parameters #
#########################


# Choose a model to test
custom_model = None # Use the standard UNet
# custom_model = "CustomUNet" # Use a model defined in ../custom_model.py

# The path were the weights should be restored from
restore_path = "../../model_weights/model_weights_family_mother_machine/model_weights_ecoli_phase.h5"

# The data for the testing
image_size = (128, 128, 1)

# You can provide a list of files or a glob(star) expression with the pathlib
# E.g. this loads all images of all bacteria in the GFP channel
eval_files = list(Path("../midap_training/").glob("**/GFP/**/*raw.tif"))

In [ ]:
# load the data 
###############

tf_pipe = TFPipeFamilyMachine(eval_files, image_size=image_size)

In [ ]:
# Load the model
################

if custom_model is None:
    # load the standard UNet
    from midap.networks.unets import UNetv1 as ModelClass
else:
    # Load the class that was defined above
    spec = importlib.util.spec_from_file_location("module", "../custom_model.py")
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    ModelClass = getattr(module, "CustomUNet")
    
# initialize the model
model = ModelClass(input_size=image_size, dropout=0.5)

# restore the weights
model.load_weights(restore_path)

In [ ]:
# Test the model
evaluation = model.evaluate(x=tf_pipe.dset_test)

In [ ]:
# plot overlays
###############

# slect the indices of the test set to plot
indices_to_plot = [11, 24, 36, 113, 148]

# get the daset as list
numpy_dset = list(tf_pipe.dset_test.unbatch().as_numpy_iterator())

for index in indices_to_plot:
    # eval the model (it's index, tuple, i, w, l)
    sample = numpy_dset[index][0][0][...,0]
    w = numpy_dset[index][0][1][...,0]
    l = numpy_dset[index][0][2][...,0]
    pred = model.predict((sample[None,...,None], w[None,...,None], l[None,...,None]))[0,...,0]
    
    # create the plot
    fig, axes = plt.subplots(1,2)
    
    # overlay with original image
    axes[0].set_title(f"Element #{index} image")
    axes[0].imshow(sample)
    axes[0].contour(pred, [0.5], colors='r', linewidths=0.5)
    axes[0].set_xticks([])
    axes[0].set_yticks([])
    
    # overlay with label
    axes[1].set_title(f"Element #{index} label")
    axes[1].imshow(l)
    axes[1].contour(pred, [0.5], colors='r', linewidths=0.5)
    axes[1].set_xticks([])
    axes[1].set_yticks([])
    plt.show()

### Eval on raw image Data

Here you can eval the network on any raw image data.

In [ ]:
# define the file path you want to eval
file_path = eval_files[1]

In [ ]:
# preprocessing
img = TFPipe.scale_pixel_vals(io.imread(file_path, as_gray=True))
row_shape, col_shape = img.shape

# get the new shape
div = 16
new_shape = (int(np.ceil(img.shape[0] / div) * div),
             int(np.ceil(img.shape[1] / div) * div))

# get the padded image
img_pad = np.pad(img, [[0, new_shape[0] - row_shape], [0, new_shape[1] - col_shape]], mode="reflect")

# add batch and channel dim
img_pad = img_pad[None,...,None]

In [ ]:
# reload the model with new config
model = ModelClass(input_size=img_pad.shape[1:], dropout=0.5, inference=True)

# restore the weights
model.load_weights(restore_path)

In [ ]:
# eval 
pred = model.predict(img_pad)

In [ ]:
# create the plot
fig, axes = plt.subplots(1,1)

# overlay with original image
axes.set_title(f"Eval on {file_path.name}")
axes.imshow(img_pad[0,...,0])
axes.contour(pred[0,...,0], [0.5], colors='r', linewidths=0.5)
axes.set_xticks([])
axes.set_yticks([])